In [1]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [6]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 41.5592,
	"longitude": -87.0652,
	"daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum", "sunshine_duration", "precipitation_probability_max", "uv_index_max", "et0_fao_evapotranspiration", "precipitation_sum", "precipitation_hours", "wind_direction_10m_dominant", "snowfall_sum"],
	"timezone": "auto",
	"forecast_days": 7
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
dates = daily.Time()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(3).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(4).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(5).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(6).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(7).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(8).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(9).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(10).ValuesAsNumpy()

Coordinates 41.561458587646484°N -87.07964324951172°E
Elevation 202.0 m asl
Timezone b'America/Chicago'b'GMT-5'
Timezone difference to GMT+0 -18000 s


In [16]:
daily_time = range(daily.Time(), daily.TimeEnd(), daily.Interval())

In [17]:
daily_time

range(1742274000, 1742878800, 86400)

In [ ]:
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)